In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

In [2]:
# parameters
customer_id = '134'
formatted_attribute = 'belt_width_inches'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Belt Pulley"""

# regex_pattern = r"(?i)([^\.]\d* in\w* Belt Width)|(\d*\.\d* in\w* Belt Width)|(\d*\-\d*\/\d* in\w* Belt Width)|(\d*\.\d in\w* Belt Width)"
regex_pattern = r"(?i)([^\.]\d* in Belt Width)|(\d*\/\d* in Belt Width)|(\d*\.\d* in Belt Width)|(\d* in Belt Width)"


value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
metric = ' Inches'

range_string = """
 0 - 1 Inches
 1.1 - 2 Inches
 2.1 - 3 Inches
 3.1 - 4 Inches
 4.1 - 5 Inches
 Greater than 5 Inches
"""

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
#CHANGE SPROCKET
df = query_from_file(file_name='query/sprocket.sql', params=params) #CHANGE SPROCKET
df['external_id']=df['external_id'].astype('str')
df['external_id']

0        00710495
1        00710497
2        00710498
3        00710500
4        00710501
           ...   
13302    00713646
13303    00713908
13304    00711347
13305    00711427
13306    00711433
Name: external_id, Length: 13307, dtype: object

In [6]:
# error

In [7]:
df[df['external_id']=='00737138']

attribute         value       customer_name external_id  \
569  belt_width_inches  0 - 1 Inches  motionindustriesus    00737138   

                                                                                                            long_desc  \
569  7826-0011 Bushing Bore Timing Belt Pulley - 0.0800 in Pitch, 11 Tooth, 0.1250 in Bore Dia., 0.2500 in Belt Width   

    resolution  
569   standard

In [32]:
# # Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# kwp.extract_keywords('Timing Belt - XH')
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0        []
1        []
2        []
3        []
4        []
         ..
13302    []
13303    []
13304    []
13305    []
13306    []
Name: long_desc, Length: 13307, dtype: object

In [9]:
#Find non-blank matches
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [10]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution                 matches  
0       standard  [0.3750 in Belt Width]  
1       standard  [0.3750 in Belt Width]  
2       standard  [0.3750 in Belt Width]  
3       standard  [0.3750 in Belt Width]  
4       standard  [0.3750 in Belt Width]  
...          ...                     ...  
13302       bulk  [1.5000 in Belt Width]  
13303       bulk  [3.0000 in Belt Width]  
13304       bulk  [0.7500 in Belt Width]  
13305       bulk  [1.0000 in Belt Width]  
13306       bulk  [1.0000 in Belt Width]  

[13307 rows x 7 columns]

In [11]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

1.0000 in Belt Width    278
2.0000 in Belt Width    163
0.5000 in Belt Width    141
3.0000 in Belt Width    139
0.7500 in Belt Width    127
1.5000 in Belt Width    119
0.3750 in Belt Width     99
4.0000 in Belt Width     48
11/16 in Belt Width      31
1/4 in Belt Width        14
5.0000 in Belt Width     13
3/4 in Belt Width         9
1/2 in Belt Width         9
3/8 in Belt Width         5
8.3750 in Belt Width      5
 4 in Belt Width          4
 2 in Belt Width          4
5/16 in Belt Width        4
 6 in Belt Width          3
7/16 in Belt Width        2
7/8 in Belt Width         2
9/16 in Belt Width        2
0.2500 in Belt Width      1
3/16 in Belt Width        1
 7 in Belt Width          1
5/8 in Belt Width         1
Name: matches, dtype: int64

In [12]:
df[df['matches'] != '[]']

attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution                 matches  
0       standard  [0.3750 in Belt Width]  
1       standard  [0.3750 in Belt Width]  
2       standard  [0.3750 in Belt Width]  
3       standard  [0.3750 in Belt Width]  
4       standard  [0.3750 in Belt Width]  
...          ...                     ...  
13302       bulk  [1.5000 in Belt Width]  
13303       bulk  [3.0000 in Belt Width]  
13304       bulk  [0.7500 in Belt Width]  
13305       bulk  [1.0000 in Belt Width]  
13306       bulk  [1.0000 in Belt Width]  

[13307 rows x 7 columns]

In [13]:
# df['match_keys'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['match_keys'].explode().value_counts()
# matches = df[df['matches'].astype(str) != '[]'] #Find all non blank matches

In [14]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [15]:
na_to_gsheet(df)

<ipython-input-14-c64bc6614460>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'


succesfully saved to worksheet:`n/a` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


In [16]:
matches = df[df['matches'].astype(str) != '[]'] 
matches 

attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution                 matches  
0       standard  [0.3750 in Belt Width]  
1       standard  [0.3750 in Belt Width]  
2       standard  [0.3750 in Belt Width]  
3       standard  [0.3750 in Belt Width]  
4       standard  [0.3750 in Belt Width]  
...          ...                     ...  
13302       bulk  [1.5000 in Belt Width]  
13303       bulk  [3.0000 in Belt Width]  
13304       bulk  [0.7500 in Belt Width]  
13305       bulk  [1.0000 in Belt Width]  
13306       bulk  [1.0000 in Belt Width]  

[1225 rows x 7 columns]

In [17]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [18]:
matches['matches'].unique()

TypeError: unhashable type: 'list'

In [19]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))

<ipython-input-19-6559beaf429e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',str(measure)))


In [20]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [21]:
def remove_fraction(numbers):
    if numbers.find('/') > -1:
        numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [22]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))
matches['matches'] = matches['matches'].apply(lambda x: remove_fraction(x))
#matches['matches'].replace(r'\-','',regex=True,inplace=True)
matches['matches'].value_counts()

<ipython-input-22-f2a921bd8cac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))
<ipython-input-22-f2a921bd8cac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda x: remove_fraction(x))


'1.0000'    278
'2.0000'    163
'0.5000'    141
'3.0000'    139
'0.7500'    127
'1.5000'    119
'0.3750'     99
'4.0000'     48
0.6875       31
0.25         14
'5.0000'     13
0.75          9
0.5           9
'8.3750'      5
0.375         5
0.3125        4
'2'           4
'4'           4
'6'           3
0.4375        2
0.5625        2
0.875         2
0.625         1
0.1875        1
'0.2500'      1
'7'           1
Name: matches, dtype: int64

In [23]:
matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))
matches

<ipython-input-23-237429777105>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))


attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

      external_id  \
0        00710495   
1        00710497   
2        00710498   
3        00710500   
4        00710501   
...           ...   
13302    00713646   
13303    00713908   
13304    00711347   
13305    00711427   
13306    00711433   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution  matches  
0       standard    0.375  
1       standard    0.375  
2       standard    0.375  
3       standard    0.375  
4       standard    0.375  
...          ...      ...  
13302       bulk    1.500  
13303       bulk    3.000  
13304       bulk    0.750  
13305       bulk    1.000  
13306       bulk    1.000  

[1225 rows x 7 columns]

In [24]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 1:
        return " 0 - 1 Inches"
        
    if num >=  1.1 and num <= 2:
        return " 1.1 - 2 Inches"
        
    if num >=  2.1 and num <= 3:
        return " 2.1 - 3 Inches"
        
    if num >=  3.1 and num <= 4:
        return " 3.1 - 4 Inches"
        
    if num >=  4.1 and num <= 5:
        return " 4.1 - 5 Inches"
        
    if num >=  5:
        return " Greater than 5 Inches"
        
    else:
        return r"n/a"



In [33]:
matches[curation_col] = matches['matches'].apply(range_app)
x=matches['external_id'].to_list()
matches['external_id']=x
matches

<ipython-input-33-f418898e4589>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches[curation_col] = matches['matches'].apply(range_app)
<ipython-input-33-f418898e4589>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['external_id']=x


attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

       external_id  \
0      ="00710495"   
1      ="00710497"   
2      ="00710498"   
3      ="00710500"   
4      ="00710501"   
...            ...   
13302  ="00713646"   
13303  ="00713908"   
13304  ="00711347"   
13305  ="00711427"   
13306  ="00711433"   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution  matches Q:belt_width_inches  
0       standard    0.375        0 - 1 Inches  
1       standard    0.375        0 - 1 Inches  
2       standard    0.375        0 - 1 Inches  
3       standard    0.375        0 - 1 Inches  
4       standard    0.375        0 - 1 Inches  
...          ...      ...                 ...  
13302       bulk    1.500      1.1 - 2 Inches  
13303       bulk    3.000      2.1 - 3 Inches  
13304       bulk    0.750        0 - 1 Inches  
13305       bulk    1.000        0 - 1 Inches  
13306       bulk    1.000        0 - 1 Inches  

[1225 rows x 8 columns]

In [26]:
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

<ipython-input-26-06b9d1939325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['external_id']=matches['external_id'].apply('="{}"'.format)


attribute           value       customer_name  \
0             belt_width_inches    0 - 1 Inches  motionindustriesus   
1             belt_width_inches    0 - 1 Inches  motionindustriesus   
2             belt_width_inches    0 - 1 Inches  motionindustriesus   
3             belt_width_inches    0 - 1 Inches  motionindustriesus   
4             belt_width_inches    0 - 1 Inches  motionindustriesus   
...                         ...             ...                 ...   
13302  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13303  timing_belt_width_inches  0 - 0.5 Inches  motionindustriesus   
13304  timing_belt_width_inches             n/a  motionindustriesus   
13305  timing_belt_width_inches             n/a  motionindustriesus   
13306  timing_belt_width_inches             n/a  motionindustriesus   

       external_id  \
0      ="00710495"   
1      ="00710497"   
2      ="00710498"   
3      ="00710500"   
4      ="00710501"   
...            ...   
13302  ="00713646"   
13303  ="00713908"   
13304  ="00711347"   
13305  ="00711427"   
13306  ="00711433"   

                                                                                                                        long_desc  \
0                7823-0101 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
1                                                            Timing Belt Pulley - 0.2000 in Pitch, 10 Tooth, 0.3750 in Belt Width   
2      1054840 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.68 in OD, 0.3750 in Belt Width   
3       113401 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.94 in OD, 0.3750 in Belt Width   
4                7823-0106 Bushing Bore Timing Belt Pulley - 0.2000 in Pitch, 11 Tooth, 0.1875 in Bore Dia., 0.3750 in Belt Width   
...                                                                                                                           ...   
13302                                                       Timing Belt Pulleys - 0.5000 in Pitch, 22 Tooth, 1.5000 in Belt Width   
13303                                                       Timing Belt Pulleys - 0.5000 in Pitch, 24 Tooth, 3.0000 in Belt Width   
13304                                                       Timing Belt Pulleys - 0.3750 in Pitch, 30 Tooth, 0.7500 in Belt Width   
13305                                                       Timing Belt Pulleys - 0.3750 in Pitch, 26 Tooth, 1.0000 in Belt Width   
13306                                                       Timing Belt Pulleys - 0.3750 in Pitch, 28 Tooth, 1.0000 in Belt Width   

      resolution  matches Q:belt_width_inches  
0       standard    0.375        0 - 1 Inches  
1       standard    0.375        0 - 1 Inches  
2       standard    0.375        0 - 1 Inches  
3       standard    0.375        0 - 1 Inches  
4       standard    0.375        0 - 1 Inches  
...          ...      ...                 ...  
13302       bulk    1.500      1.1 - 2 Inches  
13303       bulk    3.000      2.1 - 3 Inches  
13304       bulk    0.750        0 - 1 Inches  
13305       bulk    1.000        0 - 1 Inches  
13306       bulk    1.000        0 - 1 Inches  

[1225 rows x 8 columns]

In [27]:
error

NameError: name 'error' is not defined

In [28]:
gc = gcloud.Gsheets()

In [29]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [30]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Pulley_Shawn.csv',index=False)

In [31]:
looks_good('Motion Industries', attribute, df, matches)

<ipython-input-30-564dbecd2fb8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'
